<a href="https://colab.research.google.com/github/djswoosh/Uni-Bits/blob/main/PonyOrm_Interactive_Mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pony

     |████████████████████████████████| 290 kB 4.2 MB/s 
  Created wheel for pony: filename=pony-0.7.14-py3-none-any.whl size=351293 sha256=14e9558701fb9b46f3f27e95a18515731b169d9586a9dceda354cafcdd2310fe
  Stored in directory: /root/.cache/pip/wheels/de/7a/8b/82a3b5a0135fc6bad188f62e1bf6c4843142bc22c144879014
Successfully built pony


In [3]:
from pony.orm import *

In [4]:
db = Database()

In [5]:
class Person(db.Entity):
  name = Required(str)
  age = Required(int)
  cars = Set ('Car')

class Car(db.Entity):
  make = Required(str)
  model = Required(str)
  owner = Required(Person)


In [6]:
show(Person)

class Person(Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(str)
    age = Required(int)
    cars = Set(Car)


In [7]:
show(Car)

class Car(Entity):
    id = PrimaryKey(int, auto=True)
    make = Required(str)
    model = Required(str)
    owner = Required(Person)


In [8]:
db.bind(provider='sqlite', filename =':memory:')

In [10]:
db.bind(provider ='sqlite', filename='database.sqlite', create_db=True)

BindingError: ignored

In [14]:
db.generate_mapping(create_tables = True)

BindingError: ignored

In [13]:
set_sql_debug(True) #To See the SQL commans that Pony sends to the database

In [15]:
p1 = Person(name = 'John', age = 20)
p2 = Person(name = 'Mary', age = 22)
p3 = Person(name = 'Bob', age = 30)
c1 = Car(make = 'Toyota', model = 'Prius', owner = p2)
c2 = Car(make = 'Ford', model = 'Explorer', owner = p3)
commit() #This will send five insert commands to the database.

GET CONNECTION FROM THE LOCAL POOL
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Person" ("name", "age") VALUES (?, ?)
['John', 20]

INSERT INTO "Person" ("name", "age") VALUES (?, ?)
['Mary', 22]

INSERT INTO "Person" ("name", "age") VALUES (?, ?)
['Bob', 30]

INSERT INTO "Car" ("make", "model", "owner") VALUES (?, ?, ?)
['Toyota', 'Prius', 2]

INSERT INTO "Car" ("make", "model", "owner") VALUES (?, ?, ?)
['Ford', 'Explorer', 3]

COMMIT


In [17]:
@db_session
def print_person_name(person_id):
  p = Person[person_id]
  print (p.name)

In [18]:
@db_session
def add_car(person_id, make, model):
  Car(make=make, model=model, owner=Person[person_id])
  #commit() will be done automatically

In [19]:
with db_session:
  p = Person(name='Kate', age=33)
  Car(make='Audi', model='R8', owner=p)
  #commit() will be done automatically
  #database session cache will be cleared automatically
  #database connection will be returned to the pool

BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Person" ("name", "age") VALUES (?, ?)
['Kate', 33]

INSERT INTO "Car" ("make", "model", "owner") VALUES (?, ?, ?)
['Audi', 'R8', 4]

COMMIT
RELEASE CONNECTION


In [20]:
select(p for p in Person if p.age > 20)

In [21]:
select(p for p in Person if p.age >20)[:]

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT "p"."id", "p"."name", "p"."age"
FROM "Person" "p"
WHERE "p"."age" > 20



[Person[2], Person[3], Person[4]]

In [22]:
select(p for p in Person).order_by(Person.name)[:2]

SELECT "p"."id", "p"."name", "p"."age"
FROM "Person" "p"
ORDER BY "p"."name"
LIMIT 2



[Person[3], Person[1]]

In [23]:
select(p for p in Person).order_by(Person.name)[:2].show()

SELECT "p"."id", "p"."name", "p"."age"
FROM "Person" "p"
ORDER BY "p"."name"
LIMIT 2

id|name|age
--+----+---
3 |Bob |30 
1 |John|20 


In [24]:
Car.select().show()

SELECT "c"."id", "c"."make", "c"."model", "c"."owner"
FROM "Car" "c"

id|make  |model   |owner    
--+------+--------+---------
1 |Toyota|Prius   |Person[2]
2 |Ford  |Explorer|Person[3]
3 |Audi  |R8      |Person[4]


In [26]:
persons = select(p for p in Person if 'o' in p.name)
for p in persons:
  print(p.name, p.age)

SELECT "p"."id", "p"."name", "p"."age"
FROM "Person" "p"
WHERE "p"."name" LIKE '%o%'

John 20
Bob 30


In [27]:
select(p.name for p in Person if p.age != 30)[:]

SELECT DISTINCT "p"."name"
FROM "Person" "p"
WHERE "p"."age" <> 30



['John', 'Mary', 'Kate']

In [28]:
select((p, count(p.cars)) for p in Person)[:]

SELECT "p"."id", COUNT(DISTINCT "car"."id")
FROM "Person" "p"
  LEFT JOIN "Car" "car"
    ON "p"."id" = "car"."owner"
GROUP BY "p"."id"



[(Person[1], 0), (Person[2], 1), (Person[3], 1), (Person[4], 1)]

In [29]:
print(max(p.age for p in Person))

SELECT MAX("p"."age")
FROM "Person" "p"

33


In [31]:
p1= Person[1]
print(p1.name)

John


In [32]:
mary = Person.get(name = 'Mary')

print(mary.age)

SELECT "id", "name", "age"
FROM "Person"
WHERE "name" = ?
LIMIT 2
['Mary']

22


In [33]:
mary.age += 1
commit()

BEGIN IMMEDIATE TRANSACTION
UPDATE "Person"
SET "age" = ?
WHERE "id" = ?
  AND "name" = ?
  AND "age" = ?
[23, 2, 'Mary', 22]

COMMIT


In [34]:
x = 25
Person.select_by_sql('SELECT * FROM Person p WHERE p.age < $x')

BEGIN IMMEDIATE TRANSACTION
SELECT * FROM Person p WHERE p.age < ?
[25]



[Person[1], Person[2]]

In [35]:
#DB SELECT ALLOWS TO WORK WITH THE DATABASE DIRECTLY AVOIDING ENTITITES

x= 20
db.select('name FROM Person WHERE age > $x')

select name FROM Person WHERE age > ?
[20]



['Mary', 'Bob', 'Kate']

In [37]:
from pony.orm.examples.estore import *

GET NEW CONNECTION
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
PRAGMA foreign_keys = false
BEGIN IMMEDIATE TRANSACTION
CREATE TABLE "Category" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT UNIQUE NOT NULL
)

CREATE TABLE "Customer" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "email" TEXT UNIQUE NOT NULL,
  "password" TEXT NOT NULL,
  "name" TEXT NOT NULL,
  "country" TEXT NOT NULL,
  "address" TEXT NOT NULL
)

CREATE TABLE "Order" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "state" TEXT NOT NULL,
  "date_created" DATETIME NOT NULL,
  "date_shipped" DATETIME,
  "date_delivered" DATETIME,
  "total_price" DECIMAL(12, 2) NOT NULL,
  "customer" INTEGER NOT NULL REFERENCES "Customer" ("id") ON DELETE CASCADE
)

CREATE INDEX "idx_order__customer" ON "Order" ("customer")

CREATE TABLE "Product" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT NOT NULL,
  "description" TEXT NOT NULL,
  "picture" BLOB,
  "price" DECIMAL(12, 2) NOT NULL,
  "quantity" INTEGER NOT

In [38]:
populate_database()

GET NEW CONNECTION
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Customer" ("email", "password", "name", "country", "address") VALUES (?, ?, ?, ?, ?)
['john@example.com', '***', 'John Smith', 'USA', 'address 1']

INSERT INTO "Customer" ("email", "password", "name", "country", "address") VALUES (?, ?, ?, ?, ?)
['matthew@example.com', '***', 'Matthew Reed', 'USA', 'address 2']

INSERT INTO "Customer" ("email", "password", "name", "country", "address") VALUES (?, ?, ?, ?, ?)
['chuanqin@example.com', '***', 'Chuan Qin', 'China', 'address 3']

INSERT INTO "Customer" ("email", "password", "name", "country", "address") VALUES (?, ?, ?, ?, ?)
['rebecca@example.com', '***', 'Rebecca Lawson', 'USA', 'address 4']

INSERT INTO "Customer" ("email", "password", "name", "country", "address") VALUES (?, ?, ?, ?, ?)
['oliver@example.com', '***', 'Oliver Blakey', 'UK', 'address 5']

INSERT INTO "Category" ("name") VALUES (?)
['Tablets']

INSERT INTO "Category" ("name") VALUES (?)
['USB Flash Drives']

INSERT

In [39]:
select((customer.country, count(customer))
  for customer in Customer). order_by(2).first()

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT "customer"."country", COUNT(DISTINCT "customer"."id")
FROM "Customer" "customer"
GROUP BY "customer"."country"
ORDER BY 2
LIMIT 1



('China', 1)